## Amazon AgentCore Bedrock コードインタープリター チュートリアル

このチュートリアルでは、AgentCore Bedrock コードインタープリターを使用して以下を行う方法を説明します：
1. サンドボックス環境のセットアップ
2. データの読み込みと分析
3. サンドボックス環境でのコード実行
4. 結果の処理と取得

## 前提条件
- Bedrock AgentCore コードインタープリターにアクセス権限を持つAWSアカウント
- コードインタープリターリソースの作成と管理に必要なIAM権限を持っている
- 必要なPythonパッケージがインストールされている（boto3とbedrock-agentcoreを含む）
- サンプルデータファイル（data.csv）
- 分析スクリプト（stats.py）


## IAM実行ロールには以下のIAMポリシーがアタッチされている必要があります

~~~ {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:CreateCodeInterpreter",
                "bedrock-agentcore:StartCodeInterpreterSession",
                "bedrock-agentcore:InvokeCodeInterpreter",
                "bedrock-agentcore:StopCodeInterpreterSession",
                "bedrock-agentcore:DeleteCodeInterpreter",
                "bedrock-agentcore:ListCodeInterpreters",
                "bedrock-agentcore:GetCodeInterpreter"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/code-interpreter*"
        }
    ]
}



## 動作原理

コード実行サンドボックスにより、エージェントは隔離された環境でコードインタープリター、シェル、ファイルシステムを使用してユーザークエリを安全に処理できます。大規模言語モデルがツール選択を支援した後、コードはこのセッション内で実行され、合成のためにユーザーまたはエージェントに返されます。

![architecture local](code-interpreter.png)

## 1. 環境のセットアップ

まず、必要なライブラリをインポートし、コードインタープリタークライアントを初期化しましょう。

In [ ]:
!pip install --upgrade -r requirements.txt

In [ ]:
from bedrock_agentcore.tools.code_interpreter_client import CodeInterpreter
import json
from typing import Dict, Any, List

# AWSリージョンを指定してコードインタープリターを初期化
code_client = CodeInterpreter('us-west-2')
code_client.start()

## 2. ローカルファイルの読み取り

サンプルデータファイルと分析スクリプトの内容を読み取ります。

In [ ]:
def read_file(file_path: str) -> str:
    """エラーハンドリング付きでファイル内容を読み取るヘルパー関数"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"エラー: ファイル '{file_path}' が見つかりませんでした。")
        return ""
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        return ""

# 両方のファイルを読み取り
data_file_content = read_file("samples/data.csv")
code_file_content = read_file("samples/stats.py")

## 3. サンドボックス環境用ファイルの準備

サンドボックス環境で作成したいファイルを定義する構造を作成します。

In [23]:
files_to_create = [
    {
        "path": "data.csv",
        "text": data_file_content
    },
    {
        "path": "stats.py",
        "text": code_file_content
    }
]

## 4. Creating Helper Function for Tool Invocation

This helper function will make it easier to call sandbox tools and handle their responses. Within an active session, you can execute code in supported languages (Python, JavaScript), access libraries based on your dependencies configuration, generate visualizations, and maintain state between executions.

In [24]:
def call_tool(tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
    """Helper function to invoke sandbox tools
    
    Args:
        tool_name (str): Name of the tool to invoke
        arguments (Dict[str, Any]): Arguments to pass to the tool
        
    Returns:
        Dict[str, Any]: JSON formatted result
    """
    response = code_client.invoke(tool_name, arguments)
    for event in response["stream"]:
        return json.dumps(event["result"])

## 5. Writing Files to Sandbox

Now we'll write our files into the sandbox environment and verify they were created successfully.

In [25]:
# Write files to sandbox
writing_files = call_tool("writeFiles", {"content": files_to_create})
print("Writing files result:")
print(writing_files)

# Verify files were created
listing_files = call_tool("listFiles", {"path": ""})
print("\nFiles in sandbox:")
print(listing_files)

Writing files result:
{"content": [{"type": "text", "text": "Successfully wrote all 2 files"}], "isError": false}

Files in sandbox:
{"content": [{"type": "resource_link", "uri": "file:///log", "name": "log", "description": "Directory"}, {"type": "resource_link", "mimeType": "text/csv", "uri": "file:///data.csv", "name": "data.csv", "description": "File"}, {"type": "resource_link", "mimeType": "text/x-python", "uri": "file:///stats.py", "name": "stats.py", "description": "File"}, {"type": "resource_link", "uri": "file:///.ipython", "name": ".ipython", "description": "Directory"}], "isError": false}


## 6. Executing Analysis

Now we'll execute our analysis script in the sandbox environment and process the results.

In [26]:
import pprint

# Execute the analysis script
code_execute_result = call_tool("executeCode", {
    "code": files_to_create[1]['text'],
    "language": "python",
    "clearContext": True
})

# Parse and display results
analysis_results = json.loads(code_execute_result)
print("Full analysis results:")
pprint.pprint(analysis_results)

print("\nStandard output from analysis:")
print(analysis_results['structuredContent']['stdout'])

Full analysis results:
{'content': [{'text': 'Name   Place  Animal   Thing\n'
                      'count       299130  299130  299130  299130\n'
                      'unique        1722      55      50      51\n'
                      'top     Lisa White  Prague    Goat  Pencil\n'
                      'freq           222    5587    6141    6058',
              'type': 'text'}],
 'isError': False,
 'structuredContent': {'executionTime': 0.7111423015594482,
                       'exitCode': 0,
                       'stderr': '',
                       'stdout': 'Name   Place  Animal   Thing\n'
                                 'count       299130  299130  299130  299130\n'
                                 'unique        1722      55      50      51\n'
                                 'top     Lisa White  Prague    Goat  Pencil\n'
                                 'freq           222    5587    6141    6058'}}

Standard output from analysis:
Name   Place  Animal   Thing
count       29

## 7. Cleanup

Finally, we'll clean up by stopping the Code Interpreter session. Once finished using a session, the session should be shopped to release resources and avoid unnecessary charges.

In [27]:
# Stop the Code Interpreter session
code_client.stop()
print("Code Interpreter session stopped successfully!")

Code Interpreter session stopped successfully!


## まとめ

このチュートリアルでは以下を学習しました：
- コードインタープリターセッションの初期化
- 分析用ファイルの読み取りと準備
- サンドボックス環境でのコード実行
- 結果の処理と表示
- リソースのクリーンアップ